# &emsp;Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.pipeline import make_pipeline

In [2]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier    
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB  
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC

# Understanding Data

## &emsp;Importing Data

### &emsp;&emsp;Submission Sample

In [3]:
submission_sample = pd.read_csv('data/gender_submission.csv', dtype={'PassengerId': np.uint16, 'Survived': np.bool})
submission_sample.head(5)

,PassengerId,Survived
0,892,False
1,893,True
2,894,False
3,895,False
4,896,True


### &emsp;&emsp;Test Data

In [23]:
test_data = pd.read_csv('data/test.csv', dtype={'PassengerId': np.uint16,
                                                'Pclass': np.uint8,
                                                'Name': 'object',
                                                'Sex': 'object',
                                                'Age': np.float32,
                                                'SibSp': np.uint8,
                                                'Parch': np.uint8,
                                                'Ticket': 'object',
                                                'Fare': np.float32,
                                                'Cabin': 'object',
                                                'Embarked': 'object'})
test_data['Sex'] = test_data['Sex'].apply(lambda x : 0 if x == 'male' else 1)
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())
test_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### &emsp;&emsp;Train Data

In [5]:
original_train_data = pd.read_csv('data/train.csv', dtype={'PassengerId': np.uint16,
                                                'Survived': np.bool,
                                                'Pclass': np.uint8,
                                                'Name': 'object',
                                                'Sex': 'object',
                                                'Age': np.float32,
                                                'SibSp': np.uint8,
                                                'Parch': np.uint8,
                                                'Ticket': 'object',
                                                'Fare': np.float32,
                                                'Cabin': 'object',
                                                'Embarked': 'object'})
original_train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250000,NaN,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283302,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925000,NaN,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.099998,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050000,NaN,S


In [6]:
train_data = original_train_data.copy()
train_data['Sex'] = train_data['Sex'].apply(lambda x : 0 if x == 'male' else 1)

## &emsp;Analysing Data

### &emsp;&emsp;Balance

In [7]:
train_data.groupby('Survived').agg({ 'PassengerId': 'count'})

,PassengerId
Survived,
False,549
True,342


#### &emsp;&emsp;&emsp;Undersampling Majority Class

In [8]:
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')
X_resampled, y_X_resampled = rus.fit_resample(train_data.drop(columns=['Survived']), train_data['Survived'])
undersampled_train_data = pd.concat([X_resampled, y_X_resampled], axis=1)

In [9]:
undersampled_train_data.groupby('Survived').agg({ 'PassengerId': 'count'})

,PassengerId
Survived,
False,342
True,342


#### &emsp;&emsp;&emsp;Oversampling Minority Class

In [10]:
ros = RandomOverSampler(random_state=42, sampling_strategy = 'minority')
X_resampled, y_X_resampled = ros.fit_resample(train_data.drop(columns = ['Survived']), train_data['Survived'])
oversampled_train_data = pd.concat([X_resampled, y_X_resampled], axis=1)

In [11]:
oversampled_train_data.groupby('Survived').agg({ 'PassengerId': 'count'})

,PassengerId
Survived,
False,549
True,549


# MODELS

## Supervisioned

In [12]:
models = '''ExtraTreeClassifier
DecisionTreeClassifier
OneClassSVM
MLPClassifier
RadiusNeighborsClassifier
KNeighborsClassifier
ClassifierChain
MultiOutputClassifier
OutputCodeClassifier
OneVsOneClassifier
OneVsRestClassifier
SGDClassifier
RidgeClassifierCV
RidgeClassifier
PassiveAggressiveClassifier
GaussianProcessClassifier
VotingClassifier
AdaBoostClassifier
GradientBoostingClassifier
BaggingClassifier
ExtraTreesClassifier
RandomForestClassifier
BernoulliNB
CalibratedClassifierCV
GaussianNB
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LinearSVC
LogisticRegression
LogisticRegressionCV
MultinomialNB
NearestCentroid
NuSVC
Perceptron
QuadraticDiscriminantAnalysis
SVC'''
models = models.split('\n')

In [ ]:
for i in range(10):
    history = []
    for model in models:
        X = train_data.select_dtypes(exclude='object').drop(columns=['Survived','PassengerId'])
        y = train_data['Survived']

        X['Age'] = X['Age'].fillna(X['Age'].mean())

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        try:
            # clf = globals()[model]()
            clf = make_pipeline(MinMaxScaler(),globals()[model]())
            if 'fit' in dir(globals()[model]()):
                clf.fit(X_train, y_train)
            elif 'partial_fit' in dir(globals()[model]()):
                clf.partial_fit(X_train, y_train)
            elif 'fit_transform' in dir(globals()[model]()):
                clf.fit_transform(X_train, y_train)
            elif 'fit_predict' in dir(globals()[model]()):
                clf.fit_predict(X_train, y_train)

            y_pred = clf.predict(X_test)

            if -1 in y_pred:
                y_pred[y_pred == -1] = 0

            history.append([model, accuracy_score(y_test, y_pred), ''])
        except TypeError as e:
            history.append([model, 0, e])
        except Exception as e:
            history.append([model, accuracy_score(y_test, y_pred), e])

    if i == 0:
        acc_df = pd.DataFrame(history, columns = ['model', 'accuracy', 'error']).sort_values(by='accuracy', ascending=False)
    else:
        acc_df = pd.merge(acc_df, pd.DataFrame(history, columns = ['model', 'accuracy', 'error'])[['model','accuracy']], on = 'model', how='left', suffixes=('', f'_{i}'))

c:\Users\Thales\Documents\python_projects\kaggle\titanic\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Thales\Documents\python_projects\kaggle\titanic\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Thales\Documents\python_projects\kaggle\titanic\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Thales\Documents\python_projects\kaggle\titanic\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum itera

In [41]:
acc_df.set_index('model', inplace=True)

In [42]:
top5_models = acc_df.drop(columns = ['error']).agg(['mean','std'], axis=1).sort_values(by='mean', ascending=False).head(5)

In [43]:
top5_models

,mean,std
model,,
GradientBoostingClassifier,0.881818,0.000000
RandomForestClassifier,0.877273,0.007107
BaggingClassifier,0.870909,0.013754
ExtraTreesClassifier,0.865909,0.008638
DecisionTreeClassifier,0.846818,0.005689


In [17]:
dict_kwargs = {
    'BC' : {'base_estimator': DecisionTreeClassifier(max_depth=1), 'n_estimators': 100},
}

In [ ]:
clf = make_pipeline(MinMaxScaler(),MLPClassifier())
clf.fit(X_train, y_train)
y_pred = clf.predict(test_data[X.columns])

In [45]:
submission = test_data[['PassengerId']].copy()
submission.loc[:,'Survived'] = y_pred
submission['Survived'] = submission['Survived'].apply(lambda x: 0 if x == False else 1)

In [46]:
submission.to_csv('submission.csv', index=False)